In [2]:
import pandas as pd

In [8]:
df = pd.read_csv('/data/groups/trifecta/jiaqiz/newspaper_tagging/tagged/tagged_handelsblad_1860_1899.tsv', sep='\t')

In [9]:
df.head()

,Unnamed: 0.1,Unnamed: 0,newspaper,year,content,pos_tags,content_tagged
0,0,0,Algemeen Handelsblad,1887,school kerk gewone vergadering gewestelijke ve...,[school]_NOUN [kerk]_NOUN [gewone]_ADJ [vergad...,school kerk gewone vergadering gewestelijke ve...
1,1,1,Algemeen Handelsblad,1887,landbouw visscherij lienden gemeld schijnt tij...,[landbouw]_NOUN [visscherij]_ADJ [lienden]_VER...,landbouw visscherij lienden gemeld schijnt tij...
2,2,2,Algemeen Handelsblad,1887,patentrecht emeester amaterdam brengt ter kens...,[patentrecht]_NOUN [emeester]_NOUN [amaterdam]...,patentrecht emeester amaterdam brengt ter kens...
3,3,3,Algemeen Handelsblad,1887,ingezonden stukken redactie gisteren verblijd ...,[ingezonden]_VERB [stukken]_NOUN [redactie]_NO...,ingezonden stukken redactie gisteren verblijd ...
4,4,4,Algemeen Handelsblad,1887,besluiten benoemingen vnuren zyn verzoek eervo...,[besluiten]_NOUN [benoemingen]_NOUN [vnuren]_N...,besluiten benoemingen vnuren zyn verzoek eervo...


In [3]:
import glob
import os

folder_path = '/data/groups/trifecta/jiaqiz/newspaper_tagging/tagged/telegraaf/'

tsv_files = glob.glob(os.path.join(folder_path, "*.tsv"))

df_list = [pd.read_csv(f, sep="\t") for f in tsv_files]
df_all = pd.concat(df_list, ignore_index=True)

expected_cols = ["year", "pos_tags"]
df_all = df_all[expected_cols]

df_all.to_csv("/data/groups/trifecta/teresa/exploration_CHR/tagged_merged_tele.tsv", sep="\t", index=False)

In [4]:
df = pd.read_csv('/data/groups/trifecta/teresa/exploration_CHR/tagged_merged_tele.tsv', sep="\t")

In [5]:
df

,year,pos_tags
0,1931,[ijshockey]_NOUN [den]_DET [spengler]_PROPN [b...
1,1931,[siemens]_NOUN [schuckert]_VERB [onderhandelin...
2,1931,[kunstrecensies]_NOUN [zoon]_NOUN [etienne]_PR...
3,1931,[verklaring]_NOUN [kerkopbouwa]_VERB [nen]_ADV...
4,1931,[fataal]_ADJ [auto]_NOUN [ongeluk]_NOUN [estel...
...,...,...
4874496,1894,[provinciale]_ADJ [staten]_NOUN [noord]_NOUN [...
4874497,1894,[munten]_NOUN [parien]_VERB [aaa]_PROPN [dito]...
4874498,1894,[overzicht]_NOUN [effectenmarkt]_VERB [april]_...
4874499,1894,[binnenland]_NOUN [kroniek]_ADJ [emigratie]_NO...


In [7]:
print(sorted(df['year'].unique()))

[1893, 1894, 1895, 1896, 1897, 1898, 1899, 1900, 1901, 1902, 1903, 1904, 1905, 1906, 1907, 1908, 1909, 1910, 1911, 1912, 1913, 1914, 1915, 1916, 1917, 1918, 1919, 1920, 1921, 1922, 1923, 1924, 1925, 1926, 1927, 1928, 1929, 1930, 1931, 1932, 1933, 1934, 1935, 1936, 1937, 1938, 1939, 1940, 1941, 1942, 1943, 1944, 1945, 1949, 1950, 1951, 1952, 1953, 1954, 1955, 1956, 1957, 1958, 1959]


In [8]:
from collections import defaultdict
import re

keywords = {
    'blanke': ['blanke', 'blanken', 'blank'],
    'creool': ['creool', 'creolen'],
    'gekleurd': ['gekleurd', 'gekleurden'],
    'halfbloed': ['halfbloed', 'halfbloeden'],
    'hottentot': ['hottentot', 'hottentotten'],
    'inboorling': ['inboorling', 'inboorlingen'],
    'indisch': ['indisch', 'indische'],
    'indo': ['indo'],
    'indiaan': ['indiaan', 'indianen'],
    'inheems': ['inheems', 'inheemsen'],
    'inlander': ['inlander', 'inlanders'],
    'kaffer': ['kaffer', 'kaffers'],
    'khoi': ['khoi'],
    'kleurling': ['kleurling', 'kleurlingen'],
    'moor': ['moor', 'moren'],
    'marron': ['marron', 'marrons'],
    'mesties': ['mesties'],
    'mulat': ['mulat', 'mulatten'],
    'neger': ['neger', 'negers', 'negerin', 'negerinnen'],
    'primitief': ['primitief', 'primitieven'],
    'wildeman': ['wildeman', 'wildemannen'],
    'barbaar': ['barbaar', 'barbaren'],
    'koeli': ['koeli', 'koelie', 'koelies'],
    'njai': ['njai']
}

variant_to_keyword = {}
for keyword, variants in keywords.items():
    for variant in variants:
        variant_to_keyword[variant.lower()] = keyword.lower()

freq_per_year = defaultdict(lambda: defaultdict(lambda: {"NOUN": 0, "ADJ": 0}))

for _, row in df[["year", "pos_tags"]].dropna().iterrows():
    year = row["year"]
    matches = re.findall(r"\[([^\]]+)\]_(NOUN|ADJ)", row["pos_tags"])
    for word, pos in matches:
        word_lc = word.lower()
        if word_lc in variant_to_keyword:
            main_keyword = variant_to_keyword[word_lc]
            freq_per_year[main_keyword][year][pos] += 1

print("keyword\tyear\tNOUN_freq\tADJ_freq")
for word in keywords:
    word_lc = word.lower()
    for year in sorted(freq_per_year[word_lc].keys()):
        counts = freq_per_year[word_lc][year]
        print(f"{word}\t{year}\t{counts['NOUN']}\t{counts['ADJ']}")

keyword	year	NOUN_freq	ADJ_freq
blanke	1893	8	153
blanke	1894	2	150
blanke	1895	4	202
blanke	1896	26	247
blanke	1897	7	319
blanke	1898	20	260
blanke	1899	12	244
blanke	1900	23	364
blanke	1901	10	376
blanke	1902	1	72
blanke	1903	4	99
blanke	1904	10	223
blanke	1905	2	113
blanke	1906	7	150
blanke	1907	11	219
blanke	1908	13	303
blanke	1909	8	163
blanke	1910	5	168
blanke	1911	12	196
blanke	1912	12	348
blanke	1913	11	434
blanke	1914	11	565
blanke	1915	6	325
blanke	1916	7	302
blanke	1917	8	353
blanke	1918	3	160
blanke	1919	12	358
blanke	1920	8	357
blanke	1921	14	425
blanke	1922	26	405
blanke	1923	6	204
blanke	1924	17	486
blanke	1925	6	427
blanke	1926	31	839
blanke	1927	22	492
blanke	1928	24	606
blanke	1929	9	538
blanke	1930	12	870
blanke	1931	26	742
blanke	1932	17	764
blanke	1933	23	901
blanke	1934	20	1063
blanke	1935	22	872
blanke	1936	22	954
blanke	1937	23	982
blanke	1938	20	928
blanke	1939	9	868
blanke	1940	2	96
blanke	1941	3	365
blanke	1942	5	177
blanke	1943	1	91
blanke	1944	0	41
blanke	1

In [9]:
df_freq = pd.read_csv('/data/groups/trifecta/teresa/exploration_CHR/freq_tele.tsv', sep='\t')

In [15]:
df_freq

,keyword,year,NOUN_freq,ADJ_freq,period
0,blanke,1893,8,153,1860–1899
1,blanke,1894,2,150,1860–1899
2,blanke,1895,4,202,1860–1899
3,blanke,1896,26,247,1860–1899
4,blanke,1897,7,319,1860–1899
...,...,...,...,...,...
1235,njai,1925,0,1,1900–1939
1236,njai,1935,0,1,1900–1939
1237,njai,1939,0,1,1900–1939
1238,njai,1942,0,1,1940–1959


In [ ]:
bins = [1860, 1900, 1940, 1960]
labels = ["1860–1899", "1900–1939", "1940–1959"]

df_freq["year"] = pd.to_numeric(df["year"], errors="coerce")
df_freq["period"] = pd.cut(df["year"], bins=bins, labels=labels, right=False)

agg_df = df_freq.groupby(["keyword", "period"]).agg({
    # "NOUN_freq": "sum",
    "ADJ_freq": "sum"
}).reset_index()

# agg_df["TOTAL"] = agg_df["NOUN_freq"] + agg_df["ADJ_freq"]

table = agg_df.pivot(index="keyword", columns="period", values=["ADJ_freq"]) #.fillna(0).astype(int)) #, "ADJ_freq", "TOTAL"])

table.to_csv("/data/groups/trifecta/teresa/exploration_CHR/keyword_adj_tele.tsv", sep='\t')


In [ ]:
import pandas as pd
from collections import defaultdict, Counter
import re

df = pd.read_csv('/data/groups/trifecta/teresa/exploration_CHR/tagged_merged_tele.tsv', sep="\t")

bins = [1860, 1900, 1940, 1960]
labels = ["1860–1899", "1900–1939", "1940–1959"]


variant_to_keyword = {}
for keyword, variants in keywords.items():
    for variant in variants:
        variant_to_keyword[variant.lower()] = keyword.lower()


def get_period(year):
    if pd.isna(year):
        return None
    for i in range(len(bins) - 1):
        if bins[i] <= year < bins[i+1]:
            return labels[i]
    return None


adj_followed_by_noun_freq = defaultdict(lambda: defaultdict(Counter))
noun_preceded_by_adj_freq = defaultdict(lambda: defaultdict(Counter))

for _, row in df[["year", "pos_tags"]].dropna().iterrows():
    year = row["year"]
    period = get_period(year)
    if period is None:
        continue
    
    tokens = re.findall(r"\[([^\]]+)\]_(NOUN|ADJ)", row["pos_tags"])
    
    for i, (word, pos) in enumerate(tokens):
        word_lc = word.lower()
        if word_lc not in variant_to_keyword:
            continue
        
        keyword = variant_to_keyword[word_lc]
        
        if pos == "ADJ" and i + 1 < len(tokens):
            next_word, next_pos = tokens[i + 1]
            if next_pos == "NOUN":
                adj_followed_by_noun_freq[period][keyword][next_word.lower()] += 1
        
        if pos == "NOUN" and i - 1 >= 0:
            prev_word, prev_pos = tokens[i - 1]
            if prev_pos == "ADJ":
                noun_preceded_by_adj_freq[period][keyword][prev_word.lower()] += 1

def print_top(freq_dict, top_n=20, context_type="NOUN after ADJ"):
    print(f"\nTop {top_n} {context_type}\n")
    for period in labels:
        print(f"Period: {period}")
        for keyword in keywords.keys():
            counter = freq_dict[period][keyword]
            most_common = counter.most_common(top_n)
            if most_common:
                print(f"Keyword: {keyword}")
                for word, count in most_common:
                    print(f"  {word}: {count}")
        print("\n")

print_top(adj_followed_by_noun_freq, top_n=20, context_type="NOUN after ADJ keyword")
print_top(noun_preceded_by_adj_freq, top_n=20, context_type="ADJ before NOUN keyword")